# Remember to do it for preds
# Also, need to limit languages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import save_obj, load_obj, get_cn_url, get_cn_json

In [48]:
import pickle

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
label2id_zareian = load_obj('label2id_150_zareian_code_20211017')
pred2id = load_obj('pred2id_old')

In [4]:
label2id_zareian

{'kite': '/c/en/kite/n',
 'pant': '/c/en/trouser/n',
 'bowl': '/c/en/bowl/n',
 'laptop': '/c/en/laptop/n',
 'paper': '/c/en/paper/n',
 'motorcycle': '/c/en/motorcycle/n',
 'railing': '/c/en/railing/n',
 'chair': '/c/en/chair/n',
 'windshield': '/c/en/windshield/n',
 'tire': '/c/en/tire/n',
 'cup': '/c/en/cup/n',
 'bench': '/c/en/bench/n',
 'tail': '/c/en/tail/n',
 'bike': '/c/en/bicycle/n',
 'board': '/c/en/board/n',
 'orange': '/c/en/orange/n',
 'hat': '/c/en/hat/n',
 'finger': '/c/en/finger/n',
 'plate': '/c/en/plate/n',
 'woman': '/c/en/woman/n',
 'handle': '/c/en/handle/n',
 'branch': '/c/en/branch/n',
 'food': '/c/en/food/n',
 'bear': '/c/en/bear/n',
 'vase': '/c/en/vase/n',
 'vegetable': '/c/en/vegetable/n',
 'giraffe': '/c/en/giraffe/n',
 'desk': '/c/en/desk/n',
 'lady': '/c/en/lady/n',
 'towel': '/c/en/towel/n',
 'glove': '/c/en/glove/n',
 'bag': '/c/en/bag/n',
 'nose': '/c/en/nose/n',
 'rock': '/c/en/rock/n',
 'guy': '/c/en/guy/n',
 'shoe': '/c/en/shoe/n',
 'sneaker': '/c/en/g

In [49]:
pred2id

{'and': '/c/en/and',
 'says': '/c/en/say/v',
 'belonging to': '/c/en/belong/v',
 'over': '/c/en/over/r',
 'parked on': '/c/en/park/v',
 'growing on': '/c/en/grow/v',
 'standing on': '/c/en/stand/v',
 'made of': '/c/en/make/v',
 'attached to': '/c/en/attach/v',
 'at': '/c/en/at',
 'in': '/c/en/in/r',
 'hanging from': '/c/en/hang/v',
 'wears': '/c/en/wear/v',
 'in front of': '/c/en/ahead/r',
 'from': '/c/en/from',
 'for': '/c/en/belong/v',
 'watching': '/c/en/watch/v',
 'lying on': '/c/en/lie/v',
 'to': '/c/en/to',
 'behind': '/c/en/behind/r',
 'flying in': '/c/en/fly/v',
 'looking at': '/c/en/look/v',
 'on back of': '/c/en/on_back_of',
 'holding': '/c/en/hold/v',
 'between': '/c/en/between/r',
 'laying on': '/c/en/lie/v',
 'riding': '/c/en/ride/v',
 'has': '/c/en/have/v',
 'across': '/c/en/across/r',
 'wearing': '/c/en/wear/v',
 'walking on': '/c/en/walk/v',
 'eating': '/c/en/eat/v',
 'above': '/c/en/above/r',
 'part of': '/c/en/constitute/v',
 'walking in': '/c/en/walk/v',
 'sitting on

In [7]:
# [get_cn_url(cn_id) for cn_id in label2id_zareian.values()]

In [8]:
# label2id_zareian

In [9]:
from time import sleep

In [10]:
import requests

In [447]:
from collections import defaultdict
import warnings
# from warnings import warnings.warn, formatwarning
warnings.formatwarning = lambda msg, *args, **kwargs: f'{msg}\n'
import pandas as pd

# columns = [
#     'cn_id',
#     'edge_id',
#     'edge_start_raw',
#     'edge_end_raw',
#     'predicate',
#     'edge_start_matched',
#     'edge_pd_matched',
# ]
class EdgeBuilder:
    def __init__(self, cn_ids):
#         self.edges = pd.DataFrame(data, columns=['A', 'B', 'C']
        self.edges = []
        self.cn_ids = cn_ids
#         self.visited = defaultdict(lambda: defaultdict(str)) # avoid exact duplicates lol['start']['end'] = ['/r/isA']
#         self.visited = defaultdict(lambda: defaultdict(list(str)))
#         self.visited = {}
        self.visited = []
        self.duplicated_ids = set()

    def is_edge_visited_directed(self, edge):
        return edge['@id'] in self.visited # this reduced edges from 72 to 46 
#         e1, e2, r = edge
#         try:
#             return r in self.visited[e1][e2]
#         except KeyError:
#             return False
#         if e1 not in self.visited:
#             return False
#         if e2 not in self.visited[e1]:
#             return False
    
    def visit_edge_id(self, edge_id):
        self.visited.append(edge_id)
#         e1, e2, r = edge
#         if e1 not in self.visited:
#             self.visited[e1] = {e2:[r]}
#             return
#         if e2 not in self.visited[e1]:
#             self.visited[e1][e2] = [r]
#             return
#         if r not in self.visited[e1][e2]:
#             self.visited[e1][e2].append(r)
# #             return False
# #         if e2 not in self.visited[e1]:
# #             return False
#         self.visited[e1][e2].append(r)

    def is_edge_relevant(self, edge):
        entity_1, entity_2, _ = self.get_fact_from_response(edge)
        return entity_1 is not None and entity_2 is not None and entity_1 != entity_2

    def get_page_edges_objects(self, cn_id):
        '''Get the edges of the current cn_id'''
        response_json = get_cn_json(cn_id)
#         sleep(0.5) # 120 calls per min
        edges = response_json['edges']
#         self.add_page_edges(response_json)
        # Navigate next pages
        while 'view' in response_json and 'nextPage' in response_json['view']:
            next_page = response_json['view']['nextPage']
            response_json = get_cn_json(next_page)
            edges.extend(response_json['edges'])
            
        return edges
    
    def dfs_relevant_plus(self, cn_id, hops=0, plus=0):  #function for dfs 
        '''Adds edges to an empty tree. This version will walk only irrelevant edges and add them'''
        response_json = get_cn_json(cn_id)
        self.add_page_edges()
        e1, e2, r = self.get_fact_from_response(response_json)
        if self.is_edge_visited((e1, e2, r)):
            print(cn_id)
#             e1, e2, r = cn_id
#             e1, e2, r = get_cn_json(cn_id)
            is_plus = plus>=1 or plus==-1 # add edge if there's a positive plus or a negative plus (for irrelevant mode)
            is_relevant = self.add_one_edge_relevant((e1, e2, r), is_plus)
            if is_relevant:
                self.visited[e1][e2] = r
                if hops >= 1:
                    for child in self.get_children(cn_id):
                        self.dfs_relevant_plus(child, hops-1, plus-1)

    def find_match(self, new_edge):
        matched_ids = []
        for item in self.cn_ids:
            if new_edge == item:
                matched_ids.append(item)
            elif new_edge.startswith(item + '/'):            
                matched_ids.append(item)
        if len(matched_ids) > 1:
            raise ValueError(f'find_match: nonunique matched_ids={matched_ids}')
        return matched_ids[0] if len(matched_ids) > 0 else None

    def add_one_edge(self, edge):
        entity_1 = self.find_match(edge['start']['@id'])
        entity_2 = self.find_match(edge['end']['@id'])
        entity_predicate = edge['rel']['@id']
#         if entity_1 is not None and entity_2 is not None and entity_1 != entity_2: # check relevance before
        print(f'trying to add {(entity_1, entity_2, entity_predicate)}')
        self.edges.append((entity_1, entity_2, entity_predicate))
#         print('add_to_edges: added edge ({}, {}, {})'.format(entity_1, entity_2, entity_predicate))

    def add_one_edge_raw(self, edge_raw):
        self.edges.append(edge_raw)
#         e1, e2, r = edge_raw
# #         if entity_1 is not None and entity_2 is not None and entity_1 != entity_2: # check relevance before
#         print(f'trying to add {(e1, e2, r)}')
#         self.edges.append((e1, e2, r))
#         print('add_to_edges: added edge ({}, {}, {})'.format(entity_1, entity_2, entity_predicate))

    def get_raw_fact_from_response(self, response_json):
#         breakpoint()
        cn_id_start = response_json['start']['@id']
        cn_id_end = response_json['end']['@id']
        entity_predicate = response_json['rel']['@id']
        return cn_id_start, cn_id_end, entity_predicate

    def get_fact_from_response(self, response_json):
#         breakpoint()
        cn_id_start = response_json['start']['@id']
        entity_1 = self.find_match(cn_id_start)
        cn_id_end = response_json['end']['@id']
        entity_2 = self.find_match(cn_id_end)
        entity_predicate = response_json['rel']['@id']
        return entity_1, entity_2, entity_predicate

    def run_dfs_relevant_plus(self, hops=1, plus=0):
        # hops is for how many levels of relevant hops
        # plus is for how many levels of hops regardless of relevance.
        for cn_id in self.cn_ids:
            self.explore_cn_id(cn_id, hops, plus)
            
    def explore_cn_id(self, cn_id, hops, plus):
        edges = self.get_page_edges_objects(cn_id)
        for edge in edges:
            e1_raw, e2_raw, r = self.get_raw_fact_from_response(edge)
            
            # Ditch non-English concepts for now
            if not e1_raw.startswith('/c/en') or not e2_raw.startswith('/c/en'):
                continue
#             if edge['@id'] == '/a/[/r/PartOf/,/c/en/drawer/n/wn/artifact/,/c/en/desk/n/wn/artifact/]':
#                 breakpoint()
#             if cn_id == '/c/en/drawer/n':
#                 breakpoint()

#             if self.is_edge_visited_directed((e1_raw, e2_raw, r)):
#                 continue
#             self.visit((e1_raw, e2_raw, r))

#             print(f'explore_cn_id: cn_id={cn_id}, e1_raw={e1_raw}, e2_raw={e2_raw}')
            e1_matched = self.find_match(e1_raw)
            e2_matched = self.find_match(e2_raw)
        
#             breakpoint()
            if e1_matched is None or e2_matched is None:
                warnings.warn(eval(f"f'''e1_matched={e1_matched} is None or e2_matched={e2_matched} is None'''"))
                continue
            if e1_matched == e2_matched:
                warnings.warn(eval(f"f'''e1_matched=e2_matched={e2_matched}'''"))
                continue

            # Decide to go down the start vs the end node in this edge
            # TODO: use matched instead in checks or assignment?
            if e1_raw.startswith(cn_id):
#                 prev_node, next_node = e1_raw, e2_raw
                prev_node, next_node = e1_matched, e2_matched
            elif e2_raw.startswith(cn_id):
#                 prev_node, next_node = e2_raw, e1_raw
                prev_node, next_node = e2_matched, e1_matched
            else:
                raise ValueError(f'run_dfs_relevant_plus: Error mismatch cn_id={cn_id}, e1_raw={e1_raw}, e2={e2_raw}')
            
            
                
#             if edge['@id'] == '/a/[/r/Synonym/,/c/en/branch/n/wn/artifact/,/c/en/arm/n/wn/artifact/]':
#                 breakpoint()
#             if e1_matched =='/c/en/arm/n' or e2_matched == '/c/en/arm/n':
#                 breakpoint()
# #             if self.is_edge_visited_directed((prev_node, next_node, r)):
            
            edge_id = edge['@id']
            if self.is_edge_visited_directed(edge):
                self.duplicated_ids.add(edge_id)
                warnings.warn(eval(f"f'''skipping visited edge {edge['@id']}'''"))
                continue
#             self.visit((prev_node, next_node, r))
            
            is_relevant = bool(e1_matched) and bool(e2_matched)
#             new_edge = (e1_raw, e2_raw, r)
#             new_edge = (e1_matched, e2_matched, r)
            new_edge = {
                'cn_id': cn_id,
                'edge_id': edge_id,
                'edge_start_raw': e1_raw,
                'edge_end_raw': e2_raw,
                'predicate': r,
                'edge_start_matched': e1_matched,
                'edge_pd_matched': e2_matched,
            }
            # 8 scenarios: 
            # 1: relevant edge, no hops, no plus => x
            # 2: relevant edge, no hops, yes plus => √
            # 3: relevant edge, yes hops, no plus => √
            # 4: relevant edge, yes hops, yes plus => √
            # 5: irrelevant edge, no hops, no plus => x
            # 6: irrelevant edge, no hops, yes plus => √
            # 7: irrelevant edge, yes hops, no plus => x
            # 8: irrelevant edge, yes hops, yes plus => √
            
            if hops==0 and plus==0:
                # Covers cases 1 and 5
                continue
            elif is_relevant and hops==0 and plus>=1:
                # Covers cases 2
                print(f'cn_id={cn_id}, new_edge={new_edge}')
                self.visit_edge_id(edge_id)
                self.add_one_edge_raw(new_edge)
                self.explore_cn_id(next_node, hops, plus-1)
            elif is_relevant and hops>=1:
                # Covers cases 3 and 4
                print(f'cn_id={cn_id}, new_edge={new_edge}')
                self.visit_edge_id(edge_id)
                self.add_one_edge_raw(new_edge)
                self.explore_cn_id(next_node, hops-1, plus)
            elif not is_relevant and plus==0:
                # Covers cases 5 and 7
                continue
            elif not is_relevant and plus>=1:
                # Covers cases 6 and 8
                print(f'cn_id={cn_id}, new_edge={new_edge}')
                self.visit_edge_id(edge_id)
                self.add_one_edge_raw(new_edge)
                self.explore_cn_id(next_node, hops, plus-1)
            else:
                raise ValueError(f'other cases encountered. is_relevant={is_relevant}, hops={hops}, plus={plus}')


In [298]:
print(len(label2id_zareian)) # 150
print(len(set(label2id_zareian.values()))) # 147
# 3 labels are mapped to the same  ['/c/en/man/n', '/c/en/airplane/n', '/c/en/child/n']

150
147


In [473]:
cn_ids = sorted(list(set(label2id_zareian.values()).union(set(pred2id.values()))))
# cn_ids = sorted(list(set(label2id_zareian.values()))) #.union(set(pred2id.values())))

In [437]:
cn_ids = ['/c/en/drawer/n']

In [379]:
print(len(cn_ids))

1


In [380]:
cn_ids[0]

'/c/en/bear/n'

In [474]:
test = EdgeBuilder(cn_ids)

In [442]:
# test.get_page_edges_objects('/c/en/man/n')

In [475]:
test.run_dfs_relevant_plus(hops=1, plus=0)

e1_matched=/c/en/above/r is None or e2_matched=None is None
e1_matched=/c/en/across/r is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/across/r is None
e1_matched=None is None or e2_matched=/c/en/against is None
e1_matched=/c/en/against is None or e2_matched=None is None


cn_id=/c/en/against, new_edge={'cn_id': '/c/en/against', 'edge_id': '/a/[/r/Antonym/,/c/en/against/,/c/en/with/]', 'edge_start_raw': '/c/en/against', 'edge_end_raw': '/c/en/with', 'predicate': '/r/Antonym', 'edge_start_matched': '/c/en/against', 'edge_pd_matched': '/c/en/with'}


skipping visited edge /a/[/r/Antonym/,/c/en/against/,/c/en/with/]
e1_matched=None is None or e2_matched=/c/en/with is None
e1_matched=/c/en/with is None or e2_matched=None is None


cn_id=/c/en/against, new_edge={'cn_id': '/c/en/against', 'edge_id': '/a/[/r/RelatedTo/,/c/en/with/,/c/en/against/]', 'edge_start_raw': '/c/en/with', 'edge_end_raw': '/c/en/against', 'predicate': '/r/RelatedTo', 'edge_start_matched': '/c/en/with', 'edge_pd_matched': '/c/en/against'}


skipping visited edge /a/[/r/RelatedTo/,/c/en/with/,/c/en/against/]


...

skipping visited edge /a/[/r/IsA/,/c/en/stand/n/wn/artifact/,/c/en/table/n/wn/artifact/]
e1_matched=/c/en/stand/v is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/stand/v is None
e1_matched=e2_matched=/c/en/stand/v
e1_matched=None is None or e2_matched=/c/en/street/n is None
e1_matched=/c/en/street/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/telephone/n is None
e1_matched=/c/en/telephone/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/tire/n is None
e1_matched=/c/en/tire/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/towel/n is None
e1_matched=/c/en/towel/n is None or e2_matched=None is None
e1_matched=/c/en/tower/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/tower/n is None
e1_matched=/c/en/train/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/train/n is None
e1_matched=None is None or 

cn_id=/c/en/tree/n, new_edge={'cn_id': '/c/en/tree/n', 'edge_id': '/a/[/r/PartOf/,/c/en/trunk/n/wn/plant/,/c/en/tree/n/wn/plant/]', 'edge_start_raw': '/c/en/trunk/n/wn/plant', 'edge_end_raw': '/c/en/tree/n/wn/plant', 'predicate': '/r/PartOf', 'edge_start_matched': '/c/en/trunk/n', 'edge_pd_matched': '/c/en/tree/n'}


skipping visited edge /a/[/r/PartOf/,/c/en/trunk/n/wn/plant/,/c/en/tree/n/wn/plant/]
e1_matched=e2_matched=/c/en/tree/n
e1_matched=/c/en/trouser/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/trouser/n is None
e1_matched=None is None or e2_matched=/c/en/under is None
e1_matched=/c/en/under is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/vase/n is None
e1_matched=/c/en/vase/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/vegetable/n is None
e1_matched=/c/en/vegetable/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/vehicle/n is None
e1_matched=/c/en/vehicle/n is None or e2_matched=None is None
e1_matched=None is None or e2_matched=/c/en/wave/n is None
e1_matched=/c/en/wave/n is None or e2_matched=None is None


cn_id=/c/en/wave/n, new_edge={'cn_id': '/c/en/wave/n', 'edge_id': '/a/[/r/IsA/,/c/en/wave/n/wn/person/,/c/en/woman/n/wn/person/]', 'edge_start_raw': '/c/en/wave/n/wn/person', 'edge_end_raw': '/c/en/woman/n/wn/person', 'predicate': '/r/IsA', 'edge_start_matched': '/c/en/wave/n', 'edge_pd_matched': '/c/en/woman/n'}


skipping visited edge /a/[/r/IsA/,/c/en/wave/n/wn/person/,/c/en/woman/n/wn/person/]
e1_matched=None is None or e2_matched=/c/en/wire/n is None
e1_matched=/c/en/wire/n is None or e2_matched=None is None
e1_matched=e2_matched=/c/en/wire/n
e1_matched=None is None or e2_matched=/c/en/zebra/n is None
e1_matched=/c/en/zebra/n is None or e2_matched=None is None
e1_matched=e2_matched=/c/en/zebra/n


In [456]:
print(len(test.duplicated_ids))

102


In [476]:
new_edges = test.edges

In [376]:
len(test.edges)

0

In [479]:
import pandas as pd
df_dupes = pd.DataFrame(new_edges)

In [459]:
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)
pd.set_option('display.min_rows', 100)
pd.set_option('display.expand_frame_repr', True)

In [361]:
df_dupes[df_dupes['edge_id'] == '/a/[/r/IsA/,/c/en/bear/n/opencyc/bear_animal/,/c/en/animal/n/]']


,cn_id,edge_id,edge_start_raw,edge_end_raw,predicate,edge_start_matched,edge_pd_matched
7,/c/en/animal/n,"/a/[/r/IsA/,/c/en/bear/n/opencyc/bear_animal/,/c/en/animal/n/]",/c/en/bear/n/opencyc/bear_animal,/c/en/animal/n,/r/IsA,/c/en/bear/n,/c/en/animal/n


In [480]:
len(df_dupes)

154

In [481]:
df_unique = df_dupes[~df_dupes.duplicated()]

In [482]:
len(df_unique)

154

In [446]:
df_unique[df_unique['cn_id'] == '/c/en/bowl/n']

,cn_id,edge_id,edge_start_raw,edge_end_raw,predicate,edge_start_matched,edge_pd_matched
33,/c/en/bowl/n,"/a/[/r/RelatedTo/,/c/en/bowl/n/wn/artifact/,/c/en/food/n/wn/]",/c/en/bowl/n/wn/artifact,/c/en/food/n/wn,/r/RelatedTo,/c/en/bowl/n,/c/en/food/n


In [483]:
df_unique.to_csv('edges_154_h1_p0.csv', index=False)

In [331]:
len(df_unique['cn_id'].value_counts())

52

In [326]:
df_unique_72.groupby('cn_id').describe()

edge_id         \
                   count unique   
cn_id                             
/c/en/airplane/n       3      3   
/c/en/animal/n         6      6   
/c/en/arm/n            3      3   
/c/en/bear/n           1      1   
/c/en/bed/n            2      2   
/c/en/bench/n          4      4   
/c/en/bicycle/n        1      1   
/c/en/bird/n           3      3   
/c/en/board/n          3      3   
/c/en/book/n           2      2   
/c/en/boot/n           4      4   
/c/en/bottle/n         1      1   
/c/en/bowl/n           1      1   
/c/en/box/n            4      4   
/c/en/branch/n         1      1   
/c/en/building/n       5      5   
/c/en/bus/n            3      3   
/c/en/cabinet/n        1      1   
/c/en/car/n            2      2   
/c/en/cat/n            2      2   
/c/en/chair/n          4      4   
/c/en/child/n          1      1   
/c/en/clock/n          1      1   
/c/en/coat/n           3      3   
/c/en/counter/n        3      3   
/c/en/curtain/n        1      1   
/c/en/desk/n           3      3   
/c/en/dog/n            1      1   
/c/en/door/n           1      1   
/c/en/ear/n            2      2   
/c/en/eye/n            1      1   
/c/en/face/n           6      6   
/c/en/finger/n         2      2   
/c/en/flag/n           1      1   
/c/en/guy/n            1      1   
/c/en/gym_shoe/n       1      1   
/c/en/hand/n           1      1   
/c/en/handle/n         2      2   
/c/en/head/n           1      1   
/c/en/lady/n           1      1   
/c/en/lamp/n           2      2   
/c/en/leg/n            1      1   
/c/en/man/n            2      2   
/c/en/nose/n           1      1   
/c/en/person/n         1      1   
/c/en/pot/n            1      1   
/c/en/roof/n           1      1   
/c/en/room/n           1      1   
/c/en/screen/n         1      1   
/c/en/stand/n          1      1   
/c/en/tree/n           1      1   
/c/en/wave/n           1      1   

                                                                                                                    \
                                                                                                               top   
cn_id                                                                                                                
/c/en/airplane/n                     /a/[/r/PartOf/,/c/en/windshield/n/wn/artifact/,/c/en/airplane/n/wn/artifact/]   
/c/en/animal/n                                            /a/[/r/PartOf/,/c/en/face/n/wn/body/,/c/en/animal/n/wn/]   
/c/en/arm/n                                        /a/[/r/Synonym/,/c/en/arm/n/wn/group/,/c/en/branch/n/wn/group/]   
/c/en/bear/n      /a/[/r/IsA/,/c/en/bear/n/opencyc/dispositionally_pessimistic_securities_trader/,/c/en/person/n/]   
/c/en/bed/n                                         /a/[/r/RelatedTo/,/c/en/bed/n/wn/artifact/,/c/en/person/n/wn/]   
/c/en/bench/n                               /a/[/r/RelatedTo/,/c/en/bench/n/wn/artifact/,/c/en/people/n/wn/group/]   
/c/en/bicycle/n                           /a/[/r/Synonym/,/c/en/wheel/n/wn/artifact/,/c/en/bicycle/n/wn/artifact/]   
/c/en/bird/n                                                      /a/[/r/IsA/,/c/en/kite/n/wp/bird/,/c/en/bird/n/]   
/c/en/board/n                               /a/[/r/IsA/,/c/en/surfboard/n/wn/artifact/,/c/en/board/n/wn/artifact/]   
/c/en/book/n                                                  /a/[/r/IsA/,/c/en/curtain/n/wp/novel/,/c/en/book/n/]   
/c/en/boot/n                                /a/[/r/PartOf/,/c/en/counter/n/wn/artifact/,/c/en/boot/n/wn/artifact/]   
/c/en/bottle/n                              /a/[/r/PartOf/,/c/en/mouth/n/wn/artifact/,/c/en/bottle/n/wn/artifact/]   
/c/en/bowl/n                                         /a/[/r/RelatedTo/,/c/en/bowl/n/wn/artifact/,/c/en/food/n/wn/]   
/c/en/box/n                                        /a/[/r/IsA/,/c/en/box/n/wn/artifact/,/c/en/seat/n/wn/artifact/]   
/c/en/branch/n                                     /a/[/r/Synonym/,/c/en/leg/n/wn/sha

In [404]:
df_unique_72.groupby('cn_id').count().to_csv('counts_new.csv')

In [463]:
unique_edge_ids_new = df_unique['edge_id'].unique()
unique_edge_ids_old = load_obj('unique_edge_ids_old')

In [464]:
len(unique_edge_ids_new)

103

In [465]:
len(unique_edge_ids_old)

103

In [467]:
set_unique_edge_ids_new = set(unique_edge_ids_new)
set_unique_edge_ids_old = set(unique_edge_ids_old)


In [469]:
len(set_unique_edge_ids_new.intersection(set_unique_edge_ids_old))

103

In [470]:
len(set_unique_edge_ids_new.union(set_unique_edge_ids_old))

103

In [471]:
set_unique_edge_ids_new-set_unique_edge_ids_old

set()

In [472]:
set_unique_edge_ids_old-set_unique_edge_ids_new

set()

In [405]:
df_counts_new = pd.read_csv('counts_new.csv')
df_counts_old = pd.read_csv('counts_old.csv')

In [409]:
df_counts_new['cn_id']

0     /c/en/airplane/n
1       /c/en/animal/n
2          /c/en/arm/n
3         /c/en/bear/n
4          /c/en/bed/n
5        /c/en/bench/n
6      /c/en/bicycle/n
7         /c/en/bird/n
8        /c/en/board/n
9         /c/en/book/n
10        /c/en/boot/n
11      /c/en/bottle/n
12        /c/en/bowl/n
13         /c/en/box/n
14      /c/en/branch/n
15    /c/en/building/n
16         /c/en/bus/n
17     /c/en/cabinet/n
18         /c/en/car/n
19         /c/en/cat/n
20       /c/en/chair/n
21       /c/en/child/n
22       /c/en/clock/n
23        /c/en/coat/n
24     /c/en/counter/n
25     /c/en/curtain/n
26        /c/en/desk/n
27         /c/en/dog/n
28        /c/en/door/n
29         /c/en/ear/n
30         /c/en/eye/n
31        /c/en/face/n
32      /c/en/finger/n
33        /c/en/flag/n
34         /c/en/guy/n
35    /c/en/gym_shoe/n
36        /c/en/hand/n
37      /c/en/handle/n
38        /c/en/head/n
39        /c/en/lady/n
40        /c/en/lamp/n
41         /c/en/leg/n
42         /c/en/man/n
43        /

In [418]:
len(df_counts_new['cn_id'].unique().to_list())

52

In [424]:
len(df_counts_old['cn_id'].to_list())

85

In [427]:
cn_ids_new = df_counts_new['cn_id'].to_list()
cn_ids_old = df_counts_old['cn_id'].to_list()

In [429]:
len(set(cn_ids_new)) # 52
print(len(set(cn_ids_old))) # 82

85


In [432]:
set(cn_ids_old) - set(cn_ids_new)

{'/c/en/drawer/n',
 '/c/en/food/n',
 '/c/en/fruit/n',
 '/c/en/glove/n',
 '/c/en/hair/n',
 '/c/en/horse/n',
 '/c/en/house/n',
 '/c/en/jacket/n',
 '/c/en/kite/n',
 '/c/en/light/n',
 '/c/en/mouth/n',
 '/c/en/paw/n',
 '/c/en/people/n',
 '/c/en/plant/n',
 '/c/en/player/n',
 '/c/en/racket/n',
 '/c/en/seat/n',
 '/c/en/shelf/n',
 '/c/en/shoe/n',
 '/c/en/skateboard/n',
 '/c/en/surfboard/n',
 '/c/en/table/n',
 '/c/en/tail/n',
 '/c/en/tile/n',
 '/c/en/toilet/n',
 '/c/en/truck/n',
 '/c/en/trunk/n',
 '/c/en/umbrella/n',
 '/c/en/wheel/n',
 '/c/en/window/n',
 '/c/en/windshield/n',
 '/c/en/wing/n',
 '/c/en/woman/n'}

In [101]:
# # with open('../obj/' + name + '.pkl', 'rb') as f:
# #     edges_unique_144_20211017 = pickle.load(f)
# edges_zareian = load_obj('edges_unique_144_20211017')

# 1. new logic missing  ('/c/en/along', '/c/en/with', '/r/RelatedTo'). Need to check what's wrong with the logic. This is because of directionality. Not true. There's no '/c/en/with', '/c/en/along' in new_edges
 
# 2. But why more edges?

# 3. Missing edges ('/c/en/bag/n', '/c/en/hold/v', '/r/UsedFor'), and ('/c/en/bag/n', '/c/en/transport/v', '/r/UsedFor'),

# 4. '/a/[/r/IsA/,/c/en/bear/n/opencyc/bear_animal/,/c/en/animal/n/]' is a child of both animal and bear, so it's duplicated and never added.

# 5 Duplicated: drawer, food, 

# 6 The numbers of edge ids are the same across old and new. This means this step is correct. Just need to verify that the final graph build sums are the same.

# Redid the Zareian add edge logic with the local API endpoint

In [ ]:
duplicated_ids = [
    '/a/[/r/IsA/,/c/en/bear/n/opencyc/bear_animal/,/c/en/animal/n/]',
    '/a/[/r/RelatedTo/,/c/en/bowl/n/wn/artifact/,/c/en/food/n/wn/]',
    
]

In [88]:
edges_test = load_obj('edges_test')

In [103]:
len(edges_test)

321

In [164]:
sorted(list(set(edges_test)))

[('/c/en/against', '/c/en/along', '/r/DistinctFrom'),
 ('/c/en/against', '/c/en/with', '/r/Antonym'),
 ('/c/en/against', '/c/en/with', '/r/DistinctFrom'),
 ('/c/en/along', '/c/en/with', '/r/RelatedTo'),
 ('/c/en/and', '/c/en/to', '/r/RelatedTo'),
 ('/c/en/arm/n', '/c/en/branch/n', '/r/Synonym'),
 ('/c/en/bag/n', '/c/en/hold/v', '/r/UsedFor'),
 ('/c/en/bag/n', '/c/en/transport/v', '/r/UsedFor'),
 ('/c/en/basket/n', '/c/en/hold/v', '/r/UsedFor'),
 ('/c/en/bear/n', '/c/en/animal/n', '/r/IsA'),
 ('/c/en/bear/n', '/c/en/person/n', '/r/IsA'),
 ('/c/en/bed/n', '/c/en/hold/v', '/r/UsedFor'),
 ('/c/en/bed/n', '/c/en/person/n', '/r/RelatedTo'),
 ('/c/en/bed/n', '/c/en/plant/n', '/r/RelatedTo'),
 ('/c/en/bench/n', '/c/en/people/n', '/r/RelatedTo'),
 ('/c/en/bench/n', '/c/en/seat/n', '/r/IsA'),
 ('/c/en/bicycle/n', '/c/en/ride/v', '/r/UsedFor'),
 ('/c/en/boat/n', '/c/en/transport/v', '/r/UsedFor'),
 ('/c/en/boot/n', '/c/en/car/n', '/r/PartOf'),
 ('/c/en/boot/n', '/c/en/shoe/n', '/r/IsA'),
 ('/c/en

# Need to add preds

In [104]:
# edges_zareian_unique = sorted(list(set(edges_zareian)))

In [105]:
# edges_zareian_unique

In [80]:
len(edges_zareian_unique)

144

In [81]:
len(new_edges)

182